### 1. Global Settings

In [47]:
import sys
sys.path.append("../../")
import json
import os
import surprise
import papermill as pm
import pandas as pd
import time
import yaml
from tempfile import TemporaryDirectory

from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.evaluation.python_evaluation import rmse, precision_at_k, ndcg_at_k
from reco_utils.recommender.surprise.surprise_utils import compute_rating_predictions, compute_ranking_predictions

print("System version: {}".format(sys.version))
print("Surprise version: {}".format(surprise.__version__))

tmp_dir = TemporaryDirectory()

System version: 3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]
Surprise version: 1.0.6


### 2. Prepare Dataset
1. Download data and split into training, validation and test sets
2. Store the data sets to a local directory.

In [48]:
# Select Movielens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

In [49]:
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=["userID", "itemID", "rating"]
)

data.head()

,userID,itemID,rating
0,196,242,3.0
1,186,302,3.0
2,22,377,1.0
3,244,51,2.0
4,166,346,1.0


In [50]:
train, validation, test = python_random_split(data, [0.7, 0.15, 0.15])

In [51]:
DATA_DIR = os.path.join(tmp_dir.name, 'aml_data') 
os.makedirs(DATA_DIR, exist_ok=True)

TRAIN_FILE_NAME = "movielens_" + MOVIELENS_DATA_SIZE + "_train.pkl"
train.to_pickle(os.path.join(DATA_DIR, TRAIN_FILE_NAME))

VAL_FILE_NAME = "movielens_" + MOVIELENS_DATA_SIZE + "_val.pkl"
validation.to_pickle(os.path.join(DATA_DIR, VAL_FILE_NAME))

TEST_FILE_NAME = "movielens_" + MOVIELENS_DATA_SIZE + "_test.pkl"
test.to_pickle(os.path.join(DATA_DIR, TEST_FILE_NAME))

### 3. Prepare Hyperparameter Tuning 

We now prepare a training script [svd_training_nni.py](../../reco_utils/nni/svd_training.py) for the hyperparameter tuning, which will log our target metrics such as precision, NDCG, RMSE.
We define the arguments of the script and the search space for the hyperparameters. All the parameter values will be passed to our training script.

In [52]:
EXP_NAME = "movielens_" + MOVIELENS_DATA_SIZE + "_svd_model"
PRIMARY_METRIC = 'precision_at_k'
RATING_METRICS = ['rmse']
RANKING_METRICS = ['precision_at_k', 'ndcg_at_k']  
USERCOL = 'userID'
ITEMCOL = 'itemID'
RECOMMEND_SEEN = False
K = 10
RANDOM_STATE = 0
VERBOSE = True
NUM_EPOCHS = 30
BIASED = True

script_params = " ".join([
    '--datastore', DATA_DIR,
    '--train-datapath', TRAIN_FILE_NAME,
    '--validation-datapath', VAL_FILE_NAME,
    '--surprise-reader', 'ml-100k',
    '--rating-metrics', " ".join(RATING_METRICS),
    '--ranking-metrics', " ".join(RANKING_METRICS),
    '--usercol', USERCOL,
    '--itemcol', ITEMCOL,
    '--k', str(K),
    '--random-state', str(RANDOM_STATE),
    '--epochs', str(NUM_EPOCHS),
    '--primary-metric', PRIMARY_METRIC
])

if BIASED:
    script_params += ' --biased'
if VERBOSE:
    script_params += ' --verbose'
if RECOMMEND_SEEN:
    script_params += ' --recommend-seen'

In [53]:
# hyperparameters search space
# We do not set 'lr_all' and 'reg_all' because they will be overriden by the other lr_ and reg_ parameters

hyper_params = {
    'n_factors': {"_type": "choice", "_value": [10, 50, 100, 150, 200]},
    'init_mean': {"_type": "uniform", "_value": [-0.5, 0.5]},
    'init_std_dev': {"_type": "uniform", "_value": [0.01, 0.2]},
    'lr_bu': {"_type": "uniform", "_value": [1e-6, 0.1]}, 
    'lr_bi': {"_type": "uniform", "_value": [1e-6, 0.1]}, 
    'lr_pu': {"_type": "uniform", "_value": [1e-6, 0.1]}, 
    'lr_qi': {"_type": "uniform", "_value": [1e-6, 0.1]}, 
    'reg_bu': {"_type": "uniform", "_value": [1e-6, 1]},
    'reg_bi': {"_type": "uniform", "_value": [1e-6, 1]}, 
    'reg_pu': {"_type": "uniform", "_value": [1e-6, 1]}, 
    'reg_qi': {"_type": "uniform", "_value": [1e-6, 1]}
}

In [54]:
with open('../../reco_utils/nni/search_space_svd.json', 'w') as fp:
    json.dump(hyper_params, fp)

We also create a yaml file for the configuration of the trials and the tuning algorithm to be used (in this experiment we use TPE). The tuning trials will be executed locally on a [Standard_D16_v3 virtual machine](https://docs.microsoft.com/en-us/azure/virtual-machines/windows/sizes-general#dv3-series-1) (16 vcpus, 64 GB memory).

In [78]:
config = {
    'authorName': 'default',
    'experimentName': 'surprise_svd',
    'trialConcurrency': 8,
    'maxExecDuration': '1h',
    'maxTrialNum': 100,
    'trainingServicePlatform': 'local',
    # The path to Search Space
    'searchSpacePath': 'search_space_svd.json',
    'useAnnotation': False,
    'tuner': {
        'builtinTunerName': 'TPE',
        'classArgs': {
            #choice: maximize, minimize
            'optimize_mode': 'maximize'
        }
    },
    # The path and the running command of trial
    'trial':  {
      'command': 'python3 svd_training.py' + " " + script_params,
      'codeDir': '.',
      'gpuNum': 0
    }
}
 
with open('../../reco_utils/nni/config_svd.yml', 'w') as fp:
    fp.write(yaml.dump(config, default_flow_style=False))

### 4. Execute NNI Trials

The conda environment comes with NNI installed, which includes the command line tool `nnictl` for controlling and getting information about NNI experiments. <br>
To start the NNI trials for tuning, execute the following command: <br>
`nnictl create --config ../../reco_utils/nni/config_svd.yml` <br>
You can see the progress of the experiment by using the URL links output by the above command.

![](https://recodatasets.blob.core.windows.net/images/nn1.png)

![](https://recodatasets.blob.core.windows.net/images/nn2.png)

![](https://recodatasets.blob.core.windows.net/images/nn3.png)

### 5. Show Results

The trial with the best metric and the corresponding metrics and hyperparameters can be read from the Web UI,

![](https://recodatasets.blob.core.windows.net/images/nni4.png)

or from the JSON file created by the training script.

In [56]:
exp_directory = '/home/anargyri/nni/experiments/BTXE9wQP/trials/OaQaz/'

In [57]:
with open(os.path.join(exp_directory, "metrics.json"), "r") as fp:
    best_run_metrics = json.load(fp)

In [58]:
best_run_metrics

{'rmse': 1.0041081918007453,
 'ndcg_at_k': 0.09499804974666749,
 'precision_at_k': 0.0841320553780618}

In [59]:
with open(os.path.join(exp_directory, "parameter.cfg"), "r") as fp:
    parameter_values = json.load(fp)

In [60]:
parameter_values["parameters"]

{'n_factors': 150,
 'init_mean': -0.19913719902493227,
 'init_std_dev': 0.11618790065015394,
 'lr_bu': 0.016463102616016915,
 'lr_bi': 0.00018770097489760547,
 'lr_pu': 0.04836549541451997,
 'lr_qi': 0.04062529506784888,
 'reg_bu': 0.23422042744189878,
 'reg_bi': 0.8826509703299851,
 'reg_pu': 0.4579176210355413,
 'reg_qi': 0.8740101469424384}

Now we evaluate the metrics on the test data. To do this, we get the SVD model that was saved as `model.dump` in the training script.

In [61]:
svd = surprise.dump.load(os.path.join(exp_directory, "model.dump"))[1]

In [62]:
def compute_test_results(svd):
    test_results = {}
    predictions = compute_rating_predictions(svd, test, usercol="userID", itemcol="itemID")
    for metric in RATING_METRICS:
        test_results[metric] = eval(metric)(test, predictions)

    all_predictions = compute_ranking_predictions(svd, train, usercol="userID", itemcol="itemID", recommend_seen=RECOMMEND_SEEN)
    for metric in RANKING_METRICS:
        test_results[metric] = eval(metric)(test, all_predictions, col_prediction='prediction', k=K)
    return test_results

In [63]:
test_results_tpe = compute_test_results(svd)
print(test_results_tpe)

{'rmse': 1.0013418475790854, 'precision_at_k': 0.08496801705756932, 'ndcg_at_k': 0.0959363440067854}


### 6. More Tuning Algorithms
We now apply other tuning algorithms supported by NNI to the same problem. The only change needed is in the relevant entry of the configuration file.

In [64]:
config['tuner']['builtinTunerName'] = 'Random'
 
with open('../../reco_utils/nni/config_svd.yml', 'w') as fp:
    fp.write(yaml.dump(config, default_flow_style=False))

`nnictl stop`<br>
`nnictl create --config ../../reco_utils/nni/config_svd.yml`

In [65]:
svd = surprise.dump.load("/home/anargyri/nni/experiments/uOwanZo5/trials/aaju7/model.dump")[1]
test_results_random = compute_test_results(svd)

In [66]:
config['tuner']['builtinTunerName'] = 'Anneal'
 
with open('../../reco_utils/nni/config_svd.yml', 'w') as fp:
    fp.write(yaml.dump(config, default_flow_style=False))

`nnictl stop`<br>
`nnictl create --config ../../reco_utils/nni/config_svd.yml`

In [67]:
svd = surprise.dump.load("/home/anargyri/nni/experiments/l7q7bAak/trials/Fox0b/model.dump")[1]
test_results_anneal = compute_test_results(svd)

In [68]:
config['tuner']['builtinTunerName'] = 'Evolution'
 
with open('../../reco_utils/nni/config_svd.yml', 'w') as fp:
    fp.write(yaml.dump(config, default_flow_style=False))

`nnictl stop`<br>
`nnictl create --config ../../reco_utils/nni/config_svd.yml`

In [69]:
svd = surprise.dump.load("/home/anargyri/nni/experiments/jqSfATn9/trials/g1MQG/model.dump")[1]
test_results_evolution = compute_test_results(svd)

The SMAC tuner requires to be installed with the following command <br>
`nnictl package install --name=SMAC`

In [79]:
config['tuner']['builtinTunerName'] = 'SMAC'
 
with open('../../reco_utils/nni/config_svd.yml', 'w') as fp:
    fp.write(yaml.dump(config, default_flow_style=False))

`nnictl stop`<br>
`nnictl create --config ../../reco_utils/nni/config_svd.yml`

In [71]:
svd = surprise.dump.load("/home/anargyri/nni/experiments/VsOmUkYA/trials/LO7aG/model.dump")[1]
test_results_smac = compute_test_results(svd)

In [72]:
config['tuner']['builtinTunerName'] = 'MetisTuner'
 
with open('../../reco_utils/nni/config_svd.yml', 'w') as fp:
    fp.write(yaml.dump(config, default_flow_style=False))

`nnictl stop`<br>
`nnictl create --config ../../reco_utils/nni/config_svd.yml`

In [73]:
svd = surprise.dump.load("/home/anargyri/nni/experiments/v709Lcpk/trials/BckEP/model.dump")[1]
test_results_metis = compute_test_results(svd)

Hyperband follows a different style of configuration from other tuners. See [the NNI documentation](https://nni.readthedocs.io/en/latest/hyperbandAdvisor.html). Note that the [training script](../../reco_utils/nni/svd_training.py) needs to be adjusted as well, since each Hyperband trial receives an additional parameter `STEPS`, which corresponds to the resource allocation _r<sub>i</sub>_ in the [Hyperband algorithm](https://arxiv.org/pdf/1603.06560.pdf). In this example, we used `STEPS` in combination with `R` to determine the number of epochs that SVD will run for in every trial.

In [74]:
config['advisor'] = {
  'builtinAdvisorName': 'Hyperband',
  'classArgs': {
    'R': NUM_EPOCHS,
    'eta': 3,
    'optimize_mode': 'maximize'
  }
}

config.pop('tuner')
with open('../../reco_utils/nni/config_svd.yml', 'w') as fp:
    fp.write(yaml.dump(config, default_flow_style=False))

`nnictl stop`<br>
`nnictl create --config ../../reco_utils/nni/config_svd.yml`

In [75]:
svd = surprise.dump.load("/home/anargyri/nni/experiments/Dj90D2uB/trials/kOIu9/model.dump")[1]
test_results_hyperband = compute_test_results(svd)

In [76]:
pd.DataFrame(index=['TPE', 'Random Search', 'Annealing', 'Evolution', 'SMAC', 'Metis', 'Hyperband'],
             data=[res['precision_at_k'] for res in [test_results_tpe, test_results_random, test_results_anneal, 
                                                     test_results_evolution, test_results_smac, test_results_metis, 
                                                     test_results_hyperband]], 
             columns=['precision@{}'.format(K)]
            ).sort_values(by='precision@{}'.format(K), ascending=False)

,precision@10
Metis,0.102345
Random Search,0.089339
TPE,0.084968
Annealing,0.069510
Hyperband,0.068443
SMAC,0.064179
Evolution,0.058849


In [22]:
tmp_dir.cleanup()

### 7. Concluding Remarks

We showed how to tune **all** the hyperparameters accepted by Surprise SVD simultaneously, by utilizing the NNI toolkit. 
For example, training and evaluation of a single SVD model takes about 50 seconds on the 100k MovieLens data on a Standard D2_V2 VM. Searching through 100 different combinations of hyperparameters sequentially would take about 80 minutes whereas each of the above experiments took about 10. With NNI, one can take advantage of concurrency and multiple processors on a virtual machine and can use a variety of methods to navigate efficiently through a large space of hyperparameters.

### References

* [Matrix factorization algorithms in Surprise](https://surprise.readthedocs.io/en/stable/matrix_factorization.html) 
* [Surprise SVD deep-dive notebook](../02_model/surprise_svd_deep_dive.ipynb)
* [Neural Network Intelligence toolkit](https://github.com/Microsoft/nni)